# imports

In [1]:
import requests 

from bs4 import BeautifulSoup, SoupStrainer

import pandas as pd

from time import sleep

import fuckit

## build Query

In [2]:
def build_url(query, page=1):
    return "https://www.avito.ma/fr/maroc/"+query+"?o="+str(page)

## get HTML

In [3]:
def get_html(url):
    r    = requests.get(url)
    soup =  BeautifulSoup(r.content, 'lxml')
    [s.extract() for s in soup(['iframe', 'script', 'style'])]
    return soup

## Get Items

In [4]:
# get posts on one page
def get_page_posts(sp):
    return sp.find(class_="listing").find_all(class_="item")

## Parse one post

In [21]:
# get one post informations
@fuckit
def get_post(sp):
    d = {}
    d['title'] = sp.find('h2').get_text().strip()
    d['price'] = sp.find(class_='item-price').get_text().strip()
    
    extra_info = sp.find(class_='item-info-extra')
    
    d['date'] = extra_info.find(class_='age-text').get_text().strip()
    d['location'] = extra_info.find(class_='re-text').get_text().strip()
    d['category'] = extra_info.find(class_='cg-text').get_text().strip()

    return d

## Run the crawler

In [22]:
url = build_url('voiture', 1)
index = get_html(url)
posts = get_page_posts(index)
parsed_posts = [ get_post(post) for post in posts ]

## Save Results

In [23]:
df = pd.DataFrame(parsed_posts)
df.head()

,category,date,location,price,title
0,Voitures,Aujourd'hui\n23:50,Marrakech,50 000\nDH,Suzuki Celerio -2012
1,Voitures,Aujourd'hui\n23:49,Casablanca,63 000\nDH,Peugeot 307 Cc cabriolet -2004
2,Voitures,Aujourd'hui\n23:47,Casablanca,,BMW SÉRIE 4 GRAND COUPÉ -2016
3,Voitures,Aujourd'hui\n23:46,Berrechid,68 000\nDH,Golf 4 diesel -2002
4,Voitures,Aujourd'hui\n23:45,Casablanca,95 000\nDH,PEUGEOT 301 -2016


In [ ]:
df.to_excel('posts.xlsx')